In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append("../library")
import RecSys as rs
#patch = "" # For Google CoLab
path = "../data/"

In [2]:
chefmozacccepts = pd.read_csv(path+"chefmozaccepts.csv")
chefmozcuisine = pd.read_csv(path+"chefmozcuisine.csv")
chefmozhours4 = pd.read_csv(path+"chefmozhours4.csv")
chefmozparking = pd.read_csv(path+"chefmozparking.csv")
geoplaces2 = pd.read_csv(path+"geoplaces2.csv")
rating_final = pd.read_csv(path+"rating_final.csv")
usercuisine = pd.read_csv(path+"usercuisine.csv")
userpayment = pd.read_csv(path+"userpayment.csv")
userprofile = pd.read_csv(path+"userprofile.csv")
# This will contain the list of Datas that we will 
# use for the Recommendation System
listOfData_r = [rating_final]

In [3]:
listOfData = [chefmozacccepts,chefmozcuisine,chefmozhours4,
              chefmozparking,geoplaces2,rating_final,
              usercuisine,userpayment,userprofile]

In [4]:
### We have to do the secondview clean

In [5]:
rs.replaceID(realID=132706,falseID=132584,ID='placeID',data=listOfData_r)

In [6]:
###############
# Those will be the data that we will use for the model and app
###############
userProfile_r = userprofile[['userID', 'smoker', 'drink_level','dress_preference', 
                             'ambience', 'transport', 'marital_status','hijos', 
                             'religion', 'activity']]
usercuisine_list = usercuisine.groupby('userID').apply(lambda x: list(x['Rcuisine']))
usercuisine_list = pd.DataFrame({'userID':usercuisine_list.index,'Rcuisine':usercuisine_list.values})
userProfile_r = userProfile_r.merge(usercuisine_list, how = 'left', on = 'userID')
userpayment_list= userpayment.groupby('userID').apply(lambda x: list(x['Upayment']))
userpayment_list = pd.DataFrame({'userID':userpayment_list.index,'Upayment':userpayment_list.values})
userProfile_r = userProfile_r.merge(userpayment_list, how = 'left', on = 'userID')

userGeo_r = userprofile[['userID', 'latitude', 'longitude']]

userChefmozRelation_r = rating_final
userChefmozRelation_r = userChefmozRelation_r.merge(userpayment, how='left', on='userID')
userChefmozRelation_r = userChefmozRelation_r.merge(usercuisine, how='left', on='userID')

chefmozProfile_r = geoplaces2[['placeID', 'alcohol', 'smoking_area','dress_code',
                               'accessibility', 'price', 'Rambience', 'franchise',
                               'area', 'other_services']]
chefmozProfile_r= chefmozProfile_r.merge(chefmozacccepts, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozcuisine, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozhours4, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozparking, how = 'left', on = 'placeID')

chefmozGeo_r = geoplaces2[['placeID', 'latitude', 'longitude', 'the_geom_meter',
                           'address', 'city', 'state', 'country', 'fax', 'zip']]

In [23]:
l,df = rs.checkData(listOfData,'?')

In [24]:
l

[(4, 'address'),
 (4, 'city'),
 (4, 'state'),
 (4, 'country'),
 (4, 'fax'),
 (4, 'zip'),
 (4, 'url'),
 (8, 'smoker'),
 (8, 'dress_preference'),
 (8, 'ambience'),
 (8, 'transport'),
 (8, 'marital_status'),
 (8, 'hijos'),
 (8, 'activity'),
 (8, 'budget')]

In [9]:
userprofile['userID'].dtype

dtype('O')

In [10]:
type('?')

str

In [11]:
userprofile.dtypes

userID               object
latitude            float64
longitude           float64
smoker               object
drink_level          object
dress_preference     object
ambience             object
transport            object
marital_status       object
hijos                object
birth_year            int64
interest             object
personality          object
religion             object
activity             object
color                object
weight                int64
budget               object
height              float64
dtype: object